# Creating a web app to deploy the model

In [ ]:
!pip install wandb
!pip install onnxruntime
!pip install -q gradio 

     |████████████████████████████████| 1.7 MB 5.6 MB/s 
     |████████████████████████████████| 180 kB 46.2 MB/s 
     |████████████████████████████████| 97 kB 4.6 MB/s 
     |████████████████████████████████| 140 kB 55.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=bbbc61d5bd81894a61defa2791ee482eb47727fbc1628056934ec91789ce4c99
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=3d6f3a4c8863cae4e7c294043261ce5ec0a5bf4e360779006d5d8c57bd369644
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 4.8 MB 5.4 MB/s 
     |████████████████████████████████| 979 kB 4.8 MB/s 
     |███████████████████████

In [ ]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt
from gradio import gradio as gr

## Initialize Weights and Biases

In [ ]:
os.environ["WANDB_API_KEY"] = "f38031d4fe33eb56cb0d138fc5da4115578dbf43"

In [ ]:
run = wandb.init(project='creditpredict')

wandb: Currently logged in as: rajap20 (use `wandb login --relogin` to force relogin)


## Download the model and load it

In [ ]:
ARTIFACT_NAME = 'XGBoost_credit:v2'

In [ ]:
#run = wandb.init()
artifact = run.use_artifact('rajap20/creditpredict/' + ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()

In [ ]:
#run = wandb.init()ARTIFACT_NAME
#artifact = run.use_artifact('rajap20/creditpredict/' + , type='model')
#artifact_dir = artifact.download()

In [ ]:
artifact_dir

'./artifacts/XGBoost_credit:v2'

In [ ]:
!ls -al  ./artifacts/XGBoost_credit:v2

total 872
drwxr-xr-x 2 root root   4096 Dec  7 13:18 .
drwxr-xr-x 3 root root   4096 Dec  7 13:18 ..
-rw-r--r-- 1 root root 884196 Dec  7 13:18 credit_xgboost.onnx


http://onnx.ai/sklearn-onnx/auto_examples/plot_complex_pipeline.html#example-complex-pipeline

## Columns

In [ ]:
x_columns = ['area_code', 
              'state', 'resid_type', 'net_irr', 'proc_fee',
       'asset_cost', 'loan_amt', 'emi', 'net_salary', 'roi', 'tenure',
       'age']

cat_features = ['area_code', 
                'state', 'resid_type']

num_features = list(set(x_columns) - set(cat_features)) 

## Implement the predict() function

The followng inputs need to be supplied:

['KM_Driven', 'Fuel_Type', 'age',
'Transmission', 'Owner_Type', 'Seats',
'make', 'mileage_new', 'engine_new', 'model',
'power_new', 'Location']

In [ ]:
def predict_price(area_code, state, resid_type, net_irr, proc_fee,
       asset_cost, loan_amt, emi, net_salary, roi, tenure, age):


    inputs_dict = {'area_code' : area_code,
                  'state' :  state,
                  'resid_type' :  resid_type,
                  'net_irr' :  float(net_irr),
                  'proc_fee' : float(proc_fee),
                  'asset_cost' : float(asset_cost),
                  'loan_amt' :  float(loan_amt),
                  'emi' :  float(emi),                 
                  'net_salary' : float(net_salary),
                  'tenure' : float(tenure),
                  'roi' : float(roi),
                  'age' : float(age)}

    df = pd.DataFrame(inputs_dict, index = [0])
    print(df)

    inputs = {c: df[c].values for c in df.columns}
    for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))            
  
    sess = rt.InferenceSession(artifact_dir + '/credit_xgboost.onnx')
    pred_onx = sess.run(None, inputs)

    predicted_price = float(pred_onx[1][0][1])
    return {f'Probability of default: {np.round(predicted_price, 2)}'}

## Implement UI

In [ ]:

area_code = gr.inputs.Textbox(default='3075.0', label="Area Code")
state = gr.inputs.Dropdown(list(["AP", "AS", "BR", "CG", "DL", "UP", "HA", "GJ", "HP", "CH", "JH",
       "JK", "KA", "KL", "MH", "MP", "OR", "TN", "PY", "PB", "RJ", "WB",
      "UC", "TR", "MN"]), default="AP", label="State")
tenure = gr.inputs.Number(default=0.470588	, label="Tenure")
roi = gr.inputs.Number(default=-0.793282, label="Rate of Interest")
emi = gr.inputs.Number(default=0.124252, label="EMI")
proc_fee = gr.inputs.Number(default=-0.093758, label="Processing Fee")
asset_cost = gr.inputs.Number(default=87000.0, label="Asset cost")
loan_amt = gr.inputs.Number(default=71000.0, label="Loan Amount")
resid_type = gr.inputs.Radio(["O", "R", "L"],  default="O", label = "Residence Type (O: Owned, R: Rented, L: Leased)")
age = gr.inputs.Number(default=0.306122, label="Age")
net_salary = gr.inputs.Number(default=	0.158169, label="Net Salary")
net_irr = gr.inputs.Number(default=-1.321153, label="Net Internal Rate of return")

gr.Interface(predict_price, [area_code, state, resid_type, net_irr, proc_fee, asset_cost, loan_amt, emi, net_salary, roi, tenure, age], "text", live=False).launch(debug=True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://59574.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


  area_code state resid_type  ...    tenure       roi       age
0      3000    TN          R  ...  0.588235  0.508577  0.428571

[1 rows x 12 columns]
  area_code state resid_type  ...    tenure       roi       age
0      3000    PB          R  ...  0.588235  0.508577  0.428571

[1 rows x 12 columns]
  area_code state resid_type  ...    tenure       roi       age
0      3000    PB          R  ...  0.588235  0.508577  0.428571

[1 rows x 12 columns]
  area_code state resid_type  ...    tenure       roi       age
0      3000    PB          R  ...  0.588235  0.508577  0.428571

[1 rows x 12 columns]
  area_code state resid_type  ...    tenure       roi       age
0      3000    PB          R  ...  0.588235  0.508577  0.428571

[1 rows x 12 columns]
  area_code state resid_type  ...    tenure       roi       age
0      3000    PB          R  ...  0.588235  0.508577  0.428571

[1 rows x 12 columns]
  area_code state resid_type  ...    tenure       roi       age
0      3000    PB          R  